# Intro to Data Science
## Part VI. - Model Evaluation, Hyperparameter optimization, Clustering

### Table of contents

- #### Model evaluation
    - <a href="#What-is-Model-Evaluation?">Theory</a>
    - <a href="#1.-Classification-metrics">Classification Metrics</a>
        - <a href="#a)-Accuracy">Accuracy</a>
        - <a href="#b)-Confusion-matrix">Confusion matrix</a>
        - <a href="#c)-Precision,-recall,-f1-score">Precision, Recall, F1 score</a>
        - <a href="#d)-ROC-curve">ROC curve</a>
    - <a href="#2.-Regression-metrics">Regression Metrics</a>
        - <a href="#a)-Explained-variance-score">Explained variance</a>
        - <a href="#b)-Mean-absolute-error-(MAE)">MAE</a>
        - <a href="#c)-Mean-squared-error-(MSE)">MSE</a>
    - <a href="#Cluster-Validation">Clustering Metrics</a>
    
- #### Hyperparameter optimization
    - <a href="#What-is-Hyperparameter-optimization?">Theory</a>
    - <a href="#Cross-Validation">Cross Validation</a>
        - <a href="#1.-Grid-Search-Cross-Validation">Grid Search Cross Validation</a>
        - <a href="#2.-Randomized-Search-Cross-Validation">Randomized Search Cross Validation</a>
    - <a href="#3.-Other-Hyperparameter-searching-methods">Other Hyperparameter searching methods</a>
        
- #### Clustering
    - <a href="#What-is-Clustering?">Theory</a>
    - Clustering methods
        - <a href="#1.-K-Means">K-means</a>
        - <a href="#2.-DBSCAN">DBSCAN</a>
        - <a href="#3.-Hierarchical-Clustering">Hierarchical clustering</a>
        - <a href="#4.-Spectral-Clustering">Spectral clustering</a>
        - <a href="#5.-Gaussian-Mixture-Models">Gaussian Mixture Models</a>
    - <a href="#Cluster-Validation">Cluster Validation</a>
    
---

# I. Model Evaluation

## What is Model Evaluation?

When working with machine learning algorithms, data mining techniques, or statistical models, it is essential to assess whether a model has been trained effectively. Depending on the task, various metrics are available to measure the performance of a fitted model. Beyond raw metrics, there are key concepts for comparing models. Some techniques help identify overfitting, while others determine whether one model is simpler or more generalizable than another.

## Why is Model Evaluation Important?

To find the optimal solution for a given problem, it is crucial to evaluate a model’s performance. Proper evaluation helps decide whether to continue training, adjust the preprocessing pipeline, or explore alternative models.

## Tools for Model Evaluation

- **Classification metrics**  
    - Accuracy  
    - Precision  
    - Recall  
    - Precision-Recall Curve  
    - F1 Score  
    - Confusion Matrix  
    - ROC Curve  

- **Regression metrics**  
    - Mean Absolute Error (MAE)  
    - Root Mean Squared Error (RMSE)  
    - Explained Variance Score  

- **Clustering metrics**  

- **Cross-Validation**  

- **Other techniques** (e.g., model comparison methods)  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

np.random.seed = 42

In [ ]:
X_dig, y_dig = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X_dig, y_dig,
                                                    test_size=.25,
                                                    random_state=42)

In [ ]:
nn_pipe = Pipeline([('nn', MLPClassifier(hidden_layer_sizes=(5,), random_state=42))])
nn_pipe.fit(X_train, y_train)
y_hat = nn_pipe.predict(X_test)

## 1. Classification Metrics

### a) [Accuracy](http://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score)

Accuracy measures the proportion of correctly classified samples and is defined as:

$$\text{accuracy}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples}-1} 1(\hat{y}_i = y_i)$$

where $1(x)$ is the indicator function that returns 1 if the condition is true and 0 otherwise.


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_hat)

### b) Confusion Matrix

A [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix#Table_of_confusion) provides a summary of prediction results by comparing the model’s predicted labels with the actual labels.

![Type I and II errors](./pics/confusion_matrix_explained.png)  
via [@jimgthornton](https://twitter.com/jimgthornton)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_hat)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_hat))

### c) Precision, Recall, and F1 Score

*"Intuitively, precision is the ability of the classifier not to label as positive a sample that is negative, and recall is the ability of the classifier to find all the positive samples."*  
— [scikit-learn documentation](http://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-and-f-measures)

Precision, recall, and F1 score rely on four key values from the confusion matrix:  
- **True Positives (TP)**  
- **True Negatives (TN)**  
- **False Positives (FP)**  
- **False Negatives (FN)**  

These can be visualized in the following confusion matrix:

<table border="1" width="620" bordercolor="#C0C0C0" cellspacing="0" bordercolorlight="#C0C0C0" bordercolordark="#FFFFFF">
    <tbody><tr>
      <td align="center" width="170" colspan="2" rowspan="2"><font face="Calibri">Confusion Matrix</font></td>
      <td align="center" width="191" colspan="2" bgcolor="#E3EBF2"><font face="Calibri"><b>Actual</b></font></td>
    </tr>
    <tr>
      <td align="center" width="92" bgcolor="#E3EBF2"><font face="Calibri">Positive</font></td>
      <td align="center" width="95" bgcolor="#E3EBF2"><font face="Calibri">Negative</font></td>
    </tr>
    <tr>
      <td rowspan="2" align="center" width="94" bgcolor="#E3EBF2"><font face="Calibri"><b>Predicted</b></font></td>
      <td align="center" width="72" bgcolor="#E3EBF2"><font face="Calibri">Positive</font></td>
      <td align="center" width="92"><font face="Calibri">TP</font></td>
      <td align="center" width="95"><font face="Calibri">FP</font></td>
    </tr>
    <tr>
      <td align="center" width="72" bgcolor="#E3EBF2"><font face="Calibri">Negative</font></td>
      <td align="center" width="92"><font face="Calibri">FN</font></td>
      <td align="center" width="95"><font face="Calibri">TN</font></td>
    </tr>
  </tbody></table>

- **[Precision](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)**:  
  The fraction of correctly predicted positive instances among all predicted positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.  
  $$\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$$  

- **[Recall](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)**:  
  The fraction of correctly predicted positive instances among all actual positives. The recall is intuitively the ability of the classifier to find all the positive samples.  
  $$\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$  

- **[F1 Score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)**:  
  The harmonic mean of precision and recall, balancing both metrics.  
  $$F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$  

These metrics are primarily designed for binary classification problems. However, for multi-class and multi-label problems, different averaging strategies exist:  
- **Macro averaging** (`average='macro'`): Computes the unweighted mean of the metric across all classes.  
- **Micro averaging** (`average='micro'`): Aggregates TP, FP, FN across all classes before computing the metric.  

For a detailed discussion of multi-class strategies, refer to [scikit-learn documentation](http://scikit-learn.org/stable/modules/model_evaluation.html#multiclass-and-multilabel-classification).


In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_hat, average=None)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_hat, average=None)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_hat, average=None)

### d) [ROC Curve](http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html)

*A Receiver Operating Characteristic (ROC) curve is a graphical representation of a binary classifier’s performance across different threshold settings. It plots the true positive rate (TPR) against the false positive rate (FPR).*

From the [scikit-learn user guide](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):

- **True Positive Rate (TPR) / Sensitivity**:  
  $$\text{TPR} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$  
- **False Positive Rate (FPR)**:  
  $$\text{FPR} = \frac{\text{FP}}{\text{FP} + \text{TN}}$$  

The most important metric extracted from the ROC curve is the **Area Under the Curve (AUC)**, which measures the probability that a randomly chosen positive instance is ranked higher than a randomly chosen negative instance.  

According to [Wikipedia](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve):  
*"AUC is equal to the probability that a classifier will rank a randomly chosen positive instance higher than a randomly chosen negative one."*  


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

Since ROC analysis is only applicable to binary classification, multi-class problems require transformation into a binary format:

1. Generate prediction probabilities for each class.  

In [ ]:
y_score = nn_pipe.predict_proba(X_test)
classes = np.unique(y_dig)

2. Convert multi-class labels into binary form (1 for the current class, 0 otherwise).  

In [ ]:
def onevsrest(array, label):
    return (array == label).astype(int)

3. Compute **TPR**, **FPR**, and **AUC** for each class using `roc_curve`.
>    `roc_curve` returns the __FPR__, __TPR__ and the __threshold__ arrays

In [ ]:
fpr, tpr, thres, roc_auc = {}, {}, {}, {}
for i in classes:
    fpr[i], tpr[i], thres[i] = roc_curve(onevsrest(y_test, i), y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

4. Plot all class-wise ROC curves in a single figure.

In [ ]:
mycolors = sns.color_palette('muted', n_colors=len(classes))
fig, ax = plt.subplots(figsize=(8, 8))

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

# plot ROC for random baseline classifier
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# plot ROC for each class
for cls in classes:
    label = (f'ROC curve for {cls} (area = {roc_auc[cls]:0.2f})')

    ax.plot(fpr[cls], tpr[cls], color=mycolors[cls], lw=2, label=label)
    ax.legend(loc="lower right")

## 2. Regression metrics

In [ ]:
reg_pipe = Pipeline([('reg', LogisticRegression(solver='liblinear', random_state=42))])
reg_pipe.fit(X_train, y_train)
y_hat = reg_pipe.predict(X_test)


### a) [Explained Variance Score](http://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score)

Explained variance measures how well the model accounts for the variability in the target variable. It is defined as:

$$\text{Explained Variance}(y, \hat{y}) = 1 - \frac{\text{Var}(y - \hat{y})}{\text{Var}(y)}$$

where:
- $\hat{y}$ is the predicted target output,
- $y$ is the actual target output,
- $\text{Var}$ represents variance (the square of standard deviation).  

A score close to 1 indicates a well-fitting model, while a score close to 0 suggests poor performance.

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_hat)


### b) [Mean Absolute Error (MAE)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error)

MAE is the average absolute difference between actual and predicted values. It is a risk metric corresponding to the expected value of the absolute error loss (also known as L1 loss):

$$\text{MAE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|.$$

- MAE treats all errors equally, making it an intuitive measure of model accuracy.
- Unlike squared error metrics, it does not penalize large errors disproportionately.

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_hat)

### c) [Mean Squared Error (MSE)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)

MSE measures the average squared difference between actual and predicted values:

$$\text{MSE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}} - 1} (y_i - \hat{y}_i)^2.$$

- MSE gives more weight to large errors, making it sensitive to outliers.
- It is widely used due to its mathematical properties and ease of optimization.

A widely used variant, the **Root Mean Squared Error (RMSE)**, is computed as:

$$\text{RMSE}(y, \hat{y}) = \sqrt{\text{MSE}(y, \hat{y})}.$$

- RMSE is in the same units as the target variable, making interpretation easier.
- It is more sensitive to large errors compared to MAE.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_hat)

# II. Hyperparameter Optimization

## What is Hyperparameter Optimization?

According to [Wikipedia](https://en.wikipedia.org/wiki/Hyperparameter_optimization):

> _"In the context of machine learning, **hyperparameter optimization** or **model selection** is the problem of choosing a set of hyperparameters for a learning algorithm, usually with the goal of optimizing a measure of the algorithm's performance on an independent dataset. Often, cross-validation is used to estimate this generalization performance._  

> _Hyperparameter optimization contrasts with actual learning problems, which are also often cast as optimization problems but optimize a loss function on the training set alone. In effect, learning algorithms learn parameters that model/reconstruct their inputs well, while hyperparameter optimization ensures the model does not overfit its data by tuning, e.g., regularization."_  

## Why is it Important?

To find the optimal solution for a given problem, multiple models with similar predictive or exploratory power need to be trained, and the simplest effective model should be selected. This process includes choosing models and tuning hyperparameters, which can be time-consuming and tedious when done manually.  

Automated hyperparameter optimization methods help overcome this challenge by saving time and improving results.

## Common Hyperparameter Optimization Techniques

- **Grid Search**
- **Randomized Search**
- **Bayesian Optimization**
- **Gradient-based Optimization**
- **TPOT (Tree-based Pipeline Optimization Tool)**
- **Other metaheuristic approaches**

---

## [Cross-Validation](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

To select the best model, we first need a reliable way to measure its accuracy.  

### Choosing a Validation Metric

The choice of validation metric depends on the type of task:
- For **classification**, the default metric in Scikit-learn is **accuracy**.
- For **regression**, the default metric is **$R^2$ (coefficient of determination)**.
- Other metrics can be selected from [this list](http://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules).

### Why Cross-Validation?

> _"Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that simply memorizes the labels of the training samples would achieve a perfect score but fail to generalize to unseen data. This issue is called **overfitting**."_  
> — [Scikit-learn User Guide](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance)

To prevent overfitting, we split the dataset into two parts:  
- **Training set** – Used to train the model.
- **Test set** – Used to evaluate the model’s performance.

However, a single train-test split can lead to **high variance in model performance**. To obtain a more reliable estimate of a model’s accuracy, we use **Cross-Validation (CV)**, where the dataset is split multiple times using different strategies.  

More details on different cross-validation strategies can be found [here](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators).

### Hyperparameter Tuning with Cross-Validation

A model's performance can vary significantly depending on its hyperparameters. To find the best settings, multiple models must be trained with different hyperparameters and evaluated using cross-validation.  

Cross-validation provides a good estimate of a trained model’s accuracy, but additional techniques are needed to efficiently search for the optimal hyperparameters.

---

### 1. Grid Search Cross-Validation

Grid search is a systematic approach that:
- Generates a **parameter grid** from a predefined set of values.
- Evaluates the model's accuracy for every combination of hyperparameters using cross-validation.
- Selects the combination that yields the best performance.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe_digit = Pipeline([
    ('pca', PCA(svd_solver='randomized', random_state=42)),
    ('logistic', LogisticRegression(solver='liblinear', random_state=42))
])

In [ ]:
param_grid = {
    'pca__n_components': [20, 40, 64],
    'logistic__C': np.logspace(-4, 4, 3)
}

In [ ]:
grid_search = GridSearchCV(estimator=pipe_digit, 
                           param_grid=param_grid,
                           n_jobs=-1,
                           cv=5,
                           verbose=1, 
                           return_train_score=True)

In [ ]:
grid_search.fit(X_dig, y_dig)

In [ ]:
grid_search.best_estimator_.get_params(deep=False)

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
grid_search.cv_results_

In [ ]:
score_dict = grid_search.cv_results_
hmap = pd.DataFrame({
    'mean': score_dict['mean_test_score'],
    'C': [param['logistic__C'] for param in score_dict['params']],
    'n': [param['pca__n_components'] for param in score_dict['params']]
})

In [ ]:
sns.heatmap(hmap.pivot(index='C', columns='n', values='mean'), annot=True, fmt='.3f');

### 2. Randomized Search Cross-Validation

Randomized search selects hyperparameter values randomly from predefined ranges and evaluates a fixed number of configurations using cross-validation. This approach is often more efficient than grid search, especially when the search space is large.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_search_digit = RandomizedSearchCV(
    pipe_digit,
    {
        'pca__n_components': np.linspace(1, 64, 64, dtype=int),
        'logistic__C': np.logspace(-4, 4, 30),
    },
    n_iter=30, 
    n_jobs=-1,
    cv=5,
    verbose=1,
    return_train_score=True)

In [ ]:
random_search_digit.fit(X_dig, y_dig)

In [ ]:
random_score_dict = random_search_digit.cv_results_
hmap_r = pd.DataFrame({
    'mean': random_score_dict['mean_test_score'],
    'C': [param['logistic__C'] for param in random_score_dict['params']],
    'n': [param['pca__n_components'] for param in random_score_dict['params']]
})

In [ ]:
random_search_digit.best_params_, random_search_digit.best_score_

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(hmap_r.pivot(index='C', columns='n', values='mean'), annot=True, ax=ax)

### 3. Other Hyperparameter Optimization Methods

Beyond grid and randomized search, there are several advanced methods for hyperparameter tuning:

- <a href="https://github.com/rhiever/tpot">TPOT</a> – Uses genetic algorithms to automate machine learning model selection and hyperparameter tuning.
- <a href="https://www.automl.org/automl/auto-sklearn/">auto-sklearn</a> – An automated machine learning (AutoML) library that optimizes both model selection and hyperparameters.
- <a href="http://hyperopt.github.io/hyperopt/">Hyperopt</a> – Implements Bayesian optimization for more efficient hyperparameter searches.
- <a href="https://optuna.org/">Optuna</a> – A powerful and flexible framework for defining and optimizing hyperparameters using pruning and efficient sampling.
- <a href="https://ray.io/">Ray Tune</a> – A scalable hyperparameter tuning framework supporting distributed execution and integration with multiple search algorithms.
- <a href="https://facebookresearch.github.io/nevergrad/">Nevergrad</a> – A derivative-free optimization platform developed by Facebook AI for hyperparameter tuning and black-box optimization.
- <a href="https://medium.com/rants-on-machine-learning/smarter-parameter-sweeps-or-why-grid-search-is-plain-stupid-c17d97a0e881#.aify5h22n">Why Grid Search Isn't Always the Best Choice</a> – A discussion on alternative strategies for hyperparameter optimization.

These methods offer various trade-offs in terms of efficiency, interpretability, and computational cost, making them useful for different machine learning scenarios.

---
# III. Clustering

## What is Clustering?
Clustering is an <a href="http://scikit-learn.org/stable/unsupervised_learning.html">unsupervised machine learning</a> task used to uncover hidden patterns in data.  
_"Unsupervised learning is the machine learning task of inferring a function to describe hidden structure from unlabeled data. Since the examples given to the learner are unlabeled, there is no error or reward signal to evaluate a potential solution."_ — <a href="https://en.wikipedia.org/wiki/Unsupervised_learning">Wikipedia</a>

_"Cluster analysis or clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense or another) to each other than to those in other groups (clusters)."_ — <a href="https://en.wikipedia.org/wiki/Cluster_analysis">Wikipedia</a>

## Why is it Important?
Clustering is useful when labeled data is unavailable, making it essential for exploratory data analysis and pattern recognition. It helps in:  
- **Recommender systems** – Grouping similar users or items for personalized recommendations.  
- **Anomaly detection** – Identifying fraudulent transactions, network intrusions, or defective products.  
- **Medical research** – Detecting subtypes of diseases based on patient data.  
- **Image segmentation** – Separating objects in images based on pixel similarity.  
- **Social network analysis** – Identifying communities in graphs.  

## Clustering Algorithms
Different clustering algorithms have varying assumptions about data structure, making them suited for different tasks:

- **K-Means** – A fast, centroid-based algorithm that partitions data into K clusters.
- **Affinity Propagation** – Uses message-passing to identify exemplars without a predefined number of clusters.
- **Mean-Shift** – A density-based method that finds areas of high data concentration.
- **Spectral Clustering** – Uses graph-based methods to identify clusters based on connectivity.
- **Hierarchical Clustering (Ward, Agglomerative)** – Creates a tree of nested clusters, useful for visualizing relationships.
- **DBSCAN (Density-Based Spatial Clustering)** – Identifies clusters based on density, ideal for noisy and irregular data.
- **Gaussian Mixtures (GMMs)** – Models clusters as Gaussian distributions, allowing soft clustering.
- **Birch (Balanced Iterative Reducing and Clustering using Hierarchies)** – Efficient for large datasets.
- **HDBSCAN** – An improvement over DBSCAN, automatically selecting the optimal number of clusters.
- **OPTICS (Ordering Points To Identify Clustering Structure)** – Works like DBSCAN but better for variable density.
- **Support Vector Clustering** – Uses SVMs to detect cluster boundaries.

Each method has its strengths and weaknesses depending on dataset characteristics such as density, shape, and noise level.

In [ ]:
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets import make_blobs
from sklearn.datasets import load_iris

In [ ]:
n_clusters = 3
n_samples = 1500

iris = load_iris(return_X_y=True)
noisy_circles = make_circles(n_samples=n_samples, factor=.5, noise=.05, random_state=42)
noisy_moons = make_moons(n_samples=n_samples, noise=.05, random_state=42)
blobs = make_blobs(n_samples=n_samples, random_state=42)
no_structure = np.random.rand(n_samples, 2), None

datasets = {
    'iris': iris,
    'noisy_circles': noisy_circles,
    'noisy_moons': noisy_moons,
    'blobs': blobs,
    'no_structure': no_structure
}

colors = np.array(sns.color_palette('muted', n_colors=10))

In [ ]:
def cluster_datasets(model, preprocess=None, **params):
    model = model(**params)
    results = {}
    Xs = {}
    for problem, dataset in datasets.items():
        X, y = dataset
        if preprocess:
            X = preprocess.fit_transform(X, y)
        Xs[problem] = X
        model.fit(X)
        if hasattr(model, 'labels_'):
            results[problem] = model.labels_.astype('int')
        else:
            results[problem] = model.predict(X)
    return model, Xs, results

In [ ]:
def plot(Xs, results):
    plot_num = 1
    plt.figure(figsize=(len(datasets) * 4, 4))
    for problem, X in Xs.items():
        plt.subplot(1, len(datasets), plot_num)
        plt.scatter(X[:, 0], X[:, 1], color=colors[results[problem]], edgecolors='k')
        plot_num += 1

## 1. [K-Means](http://scikit-learn.org/stable/modules/clustering.html#k-means)

<img src="pics/kmeans_convergence.gif" width=300 align="left">

K-Means clustering partitions $n$ objects into $k$ clusters, where each object belongs to the cluster with the nearest mean (centroid). It produces exactly $k$ distinct clusters that aim to maximize inter-cluster separation while minimizing intra-cluster variance. However, the optimal number of clusters ($k$) is not known beforehand and must be determined from the data.

The objective of K-Means is to minimize total intra-cluster variance, or equivalently, the sum of squared distances between each point and its assigned cluster centroid:

<img src="pics/kmeans.png" width=400>

<br style="clear:left;"/>

### **Algorithm**:
1. Select $k$ initial cluster centers randomly.
2. Assign each data point to the nearest cluster center (using Euclidean distance).
3. Compute new centroids by taking the mean of all points in each cluster.
4. Repeat steps 2 and 3 until convergence (i.e., cluster assignments no longer change or centroids stabilize).

### **Choosing the Optimal $k$**
Since K-Means requires specifying $k$ in advance, various techniques help determine the best number of clusters:
- **Elbow Method** – Plots the sum of squared errors (SSE) for different $k$ values and looks for an "elbow" where diminishing returns begin.
- **Silhouette Score** – Measures how well-separated clusters are based on intra- and inter-cluster distances.
- **Gap Statistic** – Compares clustering performance against randomly generated reference data.

### **Considerations**
- **Sensitive to Initialization** – Poor centroid initialization can lead to suboptimal solutions. Methods like **K-Means++** improve this.
- **Assumes Spherical Clusters** – Struggles with non-convex or varied-density clusters (e.g., concentric circles).
- **Scalability** – Efficient on large datasets but may struggle with very high-dimensional data.

The animation is from the <a href="https://en.wikipedia.org/wiki/K-means_clustering">Wikipedia</a>.  
The description is adapted from Dr. Saed Sayad's book <a href="http://www.saedsayad.com/clustering_kmeans.htm">__An Introduction to Data Mining__</a>.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans

In [ ]:
model, Xs, results = cluster_datasets(
    model=KMeans,
    preprocess=StandardScaler(),
    n_init='auto',
    n_clusters=3,
    random_state=42
)
plot(Xs, results)

In [ ]:
model, Xs, results = cluster_datasets(
    MiniBatchKMeans,
    preprocess=StandardScaler(),
    n_init='auto',
    n_clusters=3,
    random_state=42
)
plot(Xs, results)

### Exercise: Cluster the digits dataset!

Hints:
- read with sklearn's built-in method
- use standard scaling
- use dimension reduction (pca)
- visualize results

In case you are lost, follow [sklearn's guide](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#sphx-glr-auto-examples-cluster-plot-kmeans-digits-py).

## 2. [DBSCAN (Density-Based Spatial Clustering of Applications with Noise)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

<img src="pics/dbscan_convergence.gif" align="left" style="margin-right: 20px">

DBSCAN is a clustering algorithm that groups points based on **density** rather than assuming spherical cluster shapes (like K-Means). It can identify clusters of arbitrary shape and is robust to noise (outliers), making it suitable for real-world datasets with irregular distributions.

According to scikit-learn’s [User Guide](http://scikit-learn.org/stable/modules/clustering.html#dbscan):

> *DBSCAN views clusters as areas of high density separated by areas of low density. A cluster consists of __core samples__ (high-density points) and __non-core samples__ (border points close to core samples). Noise points (outliers) remain unclustered. The density is controlled by two parameters: `eps` (the neighborhood radius) and `min_samples` (the minimum number of points required to form a dense region). Higher `min_samples` or lower `eps` increase the density requirement for a cluster to form.*

<br style="clear:left;"/>

### **Key Concepts**
DBSCAN classifies points into three categories:

- **Core Points**: A point is a core point if at least `min_samples` points (including itself) exist within a radius of `eps`. These points form the dense regions of clusters.
- **Border Points**: A point that is **within `eps` of a core point** but does not have enough neighbors to be a core point itself.
- **Outliers (Noise Points)**: A point that is **not reachable from any core point** (i.e., it doesn’t belong to any cluster).

### **How DBSCAN Works**
1. Select an arbitrary starting point.
2. Identify its `eps`-neighborhood.
   - If the point has at least `min_samples` neighbors, it becomes a core point and forms a new cluster.
   - Otherwise, it is temporarily labeled as noise (though it might later become a border point).
3. Expand the cluster by recursively adding reachable points.
4. Repeat until all points are classified as core, border, or noise.

### **Advantages of DBSCAN**
- **No need to specify the number of clusters** (unlike K-Means).  
- **Can detect arbitrarily shaped clusters** (useful for spatial and image data).  
- **Robust to noise and outliers** (outliers remain unclustered).  

### **Limitations**
- **Choosing `eps` can be tricky**: A poorly chosen `eps` may lead to over- or under-clustering.
- **Not ideal for varying-density clusters**: Struggles when clusters have different densities.
- **Computationally expensive**: Slower on high-dimensional datasets compared to K-Means.

Animation is from <a href="https://www.programmersought.com/article/46771134743/#_Densitybased_clustering_DBSCAN_88">ProgrammerSought</a>.  
A deeper theoretical explanation can be found on [Wikipedia](https://en.wikipedia.org/wiki/DBSCAN#Preliminary).

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
model, Xs, results = cluster_datasets(
    DBSCAN,
    preprocess=StandardScaler(),
    eps=0.3,
    min_samples=3
)
plot(Xs, results)

### Exercise: Cluster the generated dataset with K-Means and DBSCAN!

1. Data generation

In [ ]:
X, y = make_blobs(random_state=170, n_samples=600, centers = 5)
rng = np.random.RandomState(42)

In [ ]:
transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.show()

2. Clustering with K-means

3. Clustering with DBSCAN

Examine and explain the clustering results!

## 3. [Hierarchical Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering)

<img src="pics/hierachical_convergence.gif" align="left" style="margin-right: 20px">

<br style="clear:left;"/>

Hierarchical clustering is a family of clustering algorithms that build nested clusters in a hierarchical structure. The result is typically represented as a **dendrogram**, a tree-like diagram where:
- The **root** represents all data points in a single cluster.
- The **leaves** represent individual data points.
- **Branches** show how clusters are merged or split at different levels.

Unlike K-Means or DBSCAN, hierarchical clustering does **not** require pre-specifying the number of clusters (`k`). Instead, the hierarchy can be cut at different levels to obtain a varying number of clusters.

### **Types of Hierarchical Clustering**
1. **Agglomerative Clustering (Bottom-Up)**
   - Each data point starts as its own cluster.
   - Clusters are **iteratively merged** based on similarity until only one cluster remains.
   - This is the most common approach and is implemented in `sklearn.cluster.AgglomerativeClustering`.

2. **Divisive Clustering (Top-Down)**
   - Starts with a **single cluster** containing all data points.
   - The cluster is **recursively split** into smaller clusters.
   - Less commonly used due to higher computational cost.

### **Linkage Criteria (Merging Strategy)**
The way clusters are merged in agglomerative clustering depends on the **linkage criterion**:

- **Ward’s Method (Variance Minimization)**  
  - Minimizes the **sum of squared differences** within clusters.
  - Produces compact, spherical clusters (similar to K-Means).
  - Generally preferred for balanced, well-separated clusters.

- **Complete Linkage (Maximum Distance)**  
  - Merges clusters that have the **smallest maximum pairwise distance** between points.
  - Tends to produce **compact** and **globular** clusters.

- **Average Linkage (Mean Distance)**  
  - Merges clusters with the **smallest average pairwise distance**.
  - A compromise between Ward’s and complete linkage.

- **Single Linkage (Minimum Distance)**  
  - Merges clusters with the **smallest minimum pairwise distance**.
  - Can lead to **chained clusters**, where points are loosely linked.

### **Pros & Cons of Hierarchical Clustering**
- **No need to specify `k`** beforehand.  
- **Dendrogram provides insights** into cluster relationships.  
- **Works well for small to medium datasets**.  
- **Computationally expensive**: O(n²) to O(n³) complexity.  
- **Sensitive to noise & outliers** (especially with single linkage).  

Hierarchical clustering can be optimized with a **connectivity matrix**, which restricts which points can be merged, reducing complexity.

For more details, see scikit-learn’s [User Guide](http://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering).  
Animation is from <a href="https://www.programmersought.com/article/46771134743/#_Hierarchical_Clustering_HC_37">ProgrammerSought</a>.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

- complete

In [ ]:
model, Xs, results = cluster_datasets(
    AgglomerativeClustering,
    preprocess=StandardScaler(),
    n_clusters=3,
    linkage='complete',
)
plot(Xs, results)

In [ ]:
from sklearn.neighbors import kneighbors_graph

In [ ]:
def cluster_connections(**params):
    results = {}
    Xs = {}
    models = {}
    for problem, dataset in datasets.items():
        X, y = dataset
        X = StandardScaler().fit_transform(X, y)
        Xs[problem] = X
        connectivity = kneighbors_graph(X, n_neighbors=10, include_self=False)
        connectivity = 0.5 * (connectivity + connectivity.T)
        model = AgglomerativeClustering(connectivity=connectivity, **params)
        model.fit(X)
        results[problem] = model.labels_.astype('int')
        models[problem] = model
    return models, Xs, results

- ward

In [ ]:
models, Xs, results = cluster_connections(
    linkage='ward',
    n_clusters=2,
)
plot(Xs, results)

- average

In [ ]:
models, Xs, results = cluster_connections(
    linkage="average",
    n_clusters=2,
)
plot(Xs, results)

### Exercise: Generating dendrograms

1. Generate small dataset

In [ ]:
X_dummy, y_dummy = make_blobs(n_samples=10, n_features=2, random_state=42)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_dummy[:, 0], X_dummy[:, 1], c=y_dummy)
for i in range(len(y_dummy)):
    ax.annotate(i, (X_dummy[i, 0], X_dummy[i, 1]))

2. Use scipy's method to generat dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(X_dummy)
dendrogram(Z);

## 4. [Spectral Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html)

Spectral Clustering is a graph-based clustering algorithm that uses the eigenvalues of a similarity matrix to perform dimensionality reduction before applying a clustering method like K-Means. Unlike traditional clustering methods that rely on geometric distances (e.g., K-Means), Spectral Clustering is particularly effective for identifying non-convex and arbitrarily shaped clusters.

### How it works:
1. Construct a similarity graph from the dataset, where nodes represent data points and edges represent pairwise similarities.
2. Compute the __graph [Laplacian](](https://en.wikipedia.org/wiki/Laplacian_matrix))__ (matrix representation of a graph), which encodes the structure of the graph.
3. Compute the __eigenvectors__ of the Laplacian and use them to embed the data in a lower-dimensional space.
4. Apply K-Means clustering in this new space to assign data points to clusters.

### Advantages:
- Can capture complex cluster structures that are not linearly separable.
- Works well when the data has a natural graph structure, such as social networks or image segmentation.

### Limitations:
- Computationally expensive for large datasets due to eigenvalue decomposition.
- Requires prior knowledge of the number of clusters.
- Sensitive to the choice of similarity function.

Spectral Clustering is widely used in applications such as image segmentation, community detection in social networks, and bioinformatics.

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
model, Xs, results = cluster_datasets(
    SpectralClustering,
    preprocess=StandardScaler(),
    n_clusters=2,
    gamma=1e1,
    random_state=42
)
plot(Xs, results)

## 5. [Gaussian Mixture Models](http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)

<img src="pics/gmm_distribution.gif" align="left" style="margin-right: 20px">

<br style="clear:left;"/>

Gaussian Mixture Models (GMM) are a probabilistic clustering method that models the data as a mixture of multiple Gaussian distributions. Unlike K-Means, which assigns each point to a single cluster, GMM provides a __soft clustering__ approach, where each point is assigned a probability of belonging to multiple clusters.  

### How it works:
1. __Assumption__: The data is generated from a mixture of multiple Gaussian distributions, each defined by a mean and covariance matrix.
2. __Expectation-Maximization (EM) Algorithm__:  
   - __Expectation step (E-step)__: Computes the probability that each data point belongs to each cluster.  
   - __Maximization step (M-step)__: Updates the parameters (means, covariances, and mixture weights) of the Gaussians to maximize the likelihood of the data.
3. The process iterates until convergence, refining cluster assignments.

### Advantages:
- Can model __elliptical clusters__ with different sizes and orientations, unlike K-Means, which assumes spherical clusters.
- Provides probabilistic cluster assignments, making it more flexible in uncertain cases.
- Can handle overlapping clusters better than hard clustering techniques.

### Limitations:
- Requires selecting the number of components (clusters) in advance.
- Can suffer from local optima, so multiple initializations may be needed.
- Computationally more expensive than K-Means due to iterative probability calculations.

GMM is widely used in applications such as speaker recognition, image segmentation, anomaly detection, and density estimation.

A nice tutorial on clustering the iris dataset with GMM can be found [here](http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_covariances.html#sphx-glr-auto-examples-mixture-plot-gmm-covariances-py).  

Image from [Wikipedia](https://en.wikipedia.org/wiki/Mixture_model).

In [ ]:
from sklearn.mixture import GaussianMixture 

In [ ]:
model, Xs, results = cluster_datasets(
    GaussianMixture,
    preprocess=StandardScaler(),
    n_components=3,
    random_state=42
)
plot(Xs, results)

### Exercise

Replicate [sklearn guide's example](https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm.html#sphx-glr-auto-examples-mixture-plot-gmm-py).

## [Cluster Validation](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation)

Evaluating the performance of a clustering algorithm is more complex than evaluating supervised learning models. Since clustering is an unsupervised task, there is no direct notion of "correct" clusters. Instead, evaluation metrics focus on assessing whether the clusters effectively capture the structure of the data.

In particular, cluster evaluation should:
- Be invariant to the absolute values of cluster labels (e.g., swapping cluster labels should not change the score).
- Measure whether the clustering reflects meaningful separations in the data.
- Determine whether data points within the same cluster are more similar than points in different clusters, based on a chosen similarity metric.

Cluster validation approaches fall into two broad categories:

### 1. Evaluation with Ground Truth  
When labeled data is available, we can compare the clustering results against known class labels to assess performance. Common evaluation metrics include:

- **[Mutual Information-Based Scores](http://scikit-learn.org/stable/modules/clustering.html#mutual-information-based-scores)**  
  Mutual Information (MI) measures the agreement between predicted cluster labels (`labels_pred`) and true class labels (`labels_true`), ignoring label permutations. Higher MI indicates better clustering performance.

- **[Homogeneity, Completeness, and V-Measure](https://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure)**  
  These metrics evaluate clustering quality using conditional entropy:
  - **Homogeneity**: Each cluster should contain only members of a single class.
  - **Completeness**: All members of a given class should be assigned to the same cluster.
  - **V-Measure**: The harmonic mean of homogeneity and completeness, providing a balanced evaluation.

### 2. Evaluation Without Ground Truth  
When no labeled data is available, clustering quality is assessed using internal metrics that measure the compactness and separation of clusters.

- **[Silhouette Coefficient](http://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient)**  
  This metric quantifies how similar a sample is to its own cluster compared to other clusters. It is defined for each sample as:

  $$
  s = \frac{b - a}{\max(a, b)}
  $$

  where:
  - \( a \) = The mean distance between a sample and all other points in the same cluster.
  - \( b \) = The mean distance between a sample and the nearest neighboring cluster.

  The silhouette score ranges from -1 (poor clustering) to 1 (well-clustered data), with values around 0 indicating overlapping clusters.

Other internal evaluation metrics include the **Calinski-Harabasz Index** and **Davies-Bouldin Index**, which measure cluster compactness and separation.

By combining multiple validation techniques, we can better assess clustering performance and choose the most suitable algorithm for a given dataset.

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
for problem in datasets.keys():
    print(problem, silhouette_score(Xs[problem], results[problem], random_state=42))

### Exercise: Clustering movies

Cluster the [movielens dataset](https://grouplens.org/datasets/movielens/latest/)!

1. Download and extract the dataset (from [here](https://grouplens.org/datasets/movielens/latest/))
2. Read the readme from the archive
3. Use the datafiles to cluster the movies!

Hints:
- in movies.csv:
    - movie genres can be extracted from genres column
    - premiere year can be extracted from the title column (eg: using `r'\((\d+)\)$'` regex)
- re module is your friend (pandas already accepts regexes in str.replace() and str.extract() methods)
- use the preprocessed file from `data/movielens.csv`